In [ ]:
import pandas as pd
import spacy
import time
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../data/avis/gen_df_clean_similarity_labelled.csv",  delimiter=',', quotechar='"', encoding='utf-8',index_col= 0)

nlp = spacy.load("fr_core_news_lg")
# increase the max text to be displayed
# pd.options.display.max_colwidth = 1000
df.head()

Visualisation messages labelisés comme services efficaces vs le reste 

Labelisation des messages "services inefficaces"

Visualisation messages labélisés "services efficaces", "services ineficaces" et "mitigés" (à la fois positifs pour services efficace et inefficace)

In [ ]:
# we need to get absolute categories
df["abs_good_com"] = df.c_good_com
df["abs_bad_com"] = df.c_bad_com
# Com

for index, line in enumerate(df.c_bad_com):
    if line == df.c_good_com.iloc[index]:
        df['abs_good_com'].iloc[index] = 0
        df['abs_bad_com'].iloc[index] = 0

In [ ]:
# we need to get absolute categories
df["abs_good_value"] = df.c_good_value
df["abs_bad_value"] = df.c_bad_value
# Com

for index, line in enumerate(df.c_bad_value):
    if line == df.c_good_value.iloc[index]:
        df['abs_good_value'].iloc[index] = 0
        df['abs_bad_value'].iloc[index] = 0

In [ ]:
# top_bank topbank
list_top = [bank for bank in df.Société.value_counts().keys()[0:10]]
df_top = df[df['Société'].isin(list_top)]

import seaborn as sns
plt.figure(figsize=(10,10))
ax =sns.countplot(x = df_top.Société, hue = df_top['c_bad_efficacy']== 1)
plt.xticks(rotation=45, ha = 'right', rotation_mode = "anchor");
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi

def pyramid_ratios(df):
    count_good_efficacy = df.abs_good_efficacy[df.abs_good_efficacy == 1].count()
    count_bad_efficacy = df.abs_bad_efficacy[df.abs_bad_efficacy == 1].count()
    count_good_com = df.abs_good_com[df.abs_good_com ==1].count()
    count_bad_com = df.abs_bad_com[df.abs_bad_com == 1].count()
    count_good_value = df.abs_good_value[df.abs_good_value == 1].count()
    count_bad_value = df.abs_bad_value[df.abs_bad_value == 1].count()

    ratio_efficacy = round(count_good_efficacy/(count_good_efficacy+count_bad_efficacy),2)
    ratio_com = round(count_good_com/(count_good_com+count_bad_com),2)
    ratio_value = round(count_good_value/(count_good_value+count_bad_value),2)
    return ratio_com,ratio_efficacy,ratio_value

ratio_com,ratio_efficacy,ratio_value = pyramid_ratios(df)


def pyramid(df, bank="all banks", ax=None):
    if bank != "all banks":
        #print(f'Focusing on bank {bank}')
        data = df[df.Société == bank]
    else:
        print("Matched all banks")
        data = df
    
    ratio_com, ratio_efficacy, ratio_value = pyramid_ratios(data)

    # Define the number of variables and their ratings
    num_vars = 3
    ratings = [ratio_com, ratio_efficacy, ratio_value]
    ratings_sum = sum(ratings)

    if ratings_sum > 2:
        data_color = "ForestGreen"
    elif ratings_sum >1.5:
        data_color = "DarkOrange"
    else:
        data_color = "IndianRed"
    # Compute angle for each axis
    angles = np.linspace(0, 2 * pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # Complete the loop

    # Rotation to level the triangle
    angles = [angle + pi / 2 for angle in angles]

    # Ratings need to be repeated at the end to close the plot
    ratings = np.concatenate((ratings, [ratings[0]]))

    # Perfect ratings for reference
    perfect_ratings = np.array([1, 1, 1, 1])

    # Check if an axis is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    
    # Set the facecolor and plot
    ax.set_facecolor('white')
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(['Communication\n', '\nEfficiency', '\nValue'], fontsize=18, color=data_color)
    
     # Hide circular lines and y-axis labels
    ax.set_yticklabels([])
    ax.yaxis.grid(False)

    # Set the limit for the radial axis
    ax.set_ylim(0, 1)

    # Define a small offset for labels to avoid overlap with high scores
    label_offset = 1.1  # You can adjust this value as needed

    ax.plot(angles, perfect_ratings, linewidth=1, linestyle=':', color='grey', alpha=0.5)
    ax.fill(angles, perfect_ratings, color='Ivory', alpha=0.5)
    # plotting the perfect triangle for visual reference
    ax.plot(angles, ratings, linewidth=2, linestyle='solid', color=data_color)
    ax.fill(angles, ratings, data_color, alpha=0.25)
    ax.spines['polar'].set_visible(False)
    ax.grid(color='grey', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.yticks([], [], color="black", size=8)
    plt.ylim(0, 1)

    # Adding values at triangle tips
    for angle, rating in zip(angles[:-1], ratings[:-1]):
        ax.text(angle, rating + 0.10, f'{rating:.2f}', ha='center', va='center', fontsize=14, color=data_color)
    
    # Adding the sum of ratings in bold at the center of the triangle
    ax.text(0, 0, f'{ratings_sum:.2f}', ha='center', va='center', fontsize=25, color= data_color, fontweight='bold')
    ax.set_title(f"Scores for {bank}", size=25)


In [ ]:
pyramid(df, "Oney")

In [ ]:
# we will only keep the banks with > 1000 reviews
all_banks =  [bank for bank in df.Société.unique()]3+
high_pass = []

for bank in all_banks:
   tally = sum(df.Société == bank)
   print(f"for {bank}, count is : {tally}")
   if tally > 1000:
      high_pass.append(bank)
high_pass

In [ ]:
# filtering for banks with over 1000 reviews
banks = high_pass  
# calculating the number of rows and cols
n = len(banks)
cols = 4  
rows = n // cols + (n % cols > 0)  

# plotting the pyramids
fig, axs = plt.subplots(rows, cols, figsize=(6 * cols, 6 * rows), subplot_kw=dict(polar=True))
axs = axs.flatten()  

fig.suptitle('Ratios of positive user sentiment regarding communication, efficacy and value of banking services for banks with > 1000 reviews', fontsize=20)  # Add an overall title

for idx, bank in enumerate(banks):
    ax = axs[idx]
    pyramid(df, bank, ax=ax)
    ax.grid(False)  
    ax.set_yticklabels([]) 
    ax.set_title(f'{bank}', fontsize=12) 

for ax in axs[len(banks):]:
    ax.set_visible(False)

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) 

plt.show()
